In [2]:
#import kagglehub

"""
# Download latest version
path = kagglehub.dataset_download("Path to working directory", "dataset_name") # Esto es para descartar el dataset inicial, poner el path en tu ordenador local

print("Path to dataset files:", path)
"""

'\n# Download latest version\npath = kagglehub.dataset_download("Path to working directory", "dataset_name") # Esto es para descartar el dataset inicial, poner el path en tu ordenador local\n\nprint("Path to dataset files:", path)\n'

In [3]:
import pandas as pd
df = pd.read_csv('amazon_reviews.csv')
df.head() #Aqui podemos ver el dataset usado, varias columnas innecesarias


,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [4]:
#Limpiar el dataset
dfSimple = df.drop(columns=['reviewerName', 'reviewTime', 'day_diff', 'helpful_yes', 'helpful_no', 'total_vote', 'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'])

#De inicio trabajaremos con un dataset masivamente simplificado para poder hacer pruebas, la columna unnamed:0 no se elimina porque es un identificador de la fila y puede ser util mas adelante
dfSimple.dropna(subset=["reviewText"], inplace=True)
dfSimple.head()


,Unnamed: 0,overall,reviewText
0,0,4.0,No issues.
1,1,5.0,"Purchased this for my device, it worked as adv..."
2,2,4.0,it works as expected. I should have sprung for...
3,3,5.0,This think has worked out great.Had a diff. br...
4,4,5.0,"Bought it with Retail Packaging, arrived legit..."


In [5]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [6]:
# NLTK resources
nltk.download('stopwords') #Llista de stopwords + comunes
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\minaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\minaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Split text into words
    words = text.split()
    # Remove stopwords and apply lemmatization
    stop_words = set(stopwords.words('english'))
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)


In [8]:
# Create a new DataFrame with cleaned reviewText and the overall column
dfCleaned = dfSimple.copy()
dfCleaned['reviewText'] = dfCleaned['reviewText'].apply(clean_text)  # Apply the cleaning function
#dfCleaned['overall'] = dfCleaned['overall']  # Include the overall column

print(dfCleaned.head())

   Unnamed: 0  overall                                         reviewText
0           0      4.0                                              issue
1           1      5.0  purchased device worked advertised never much ...
2           2      4.0  work expected sprung higher capacity think mad...
3           3      5.0  think worked greathad diff bran gb card went s...
4           4      5.0  bought retail packaging arrived legit orange e...


In [9]:
# Save the cleaned dataset 
dfCleaned.to_csv('amazon_reviews_Cleaned.csv', index=False)

print("Cleaned dataset saved as 'amazon_reviews_Cleaned.csv'")

Cleaned dataset saved as 'amazon_reviews_Cleaned.csv'


In [10]:
#Por ahora, trabajaremos con valores de 0 o 1 en la columna overall para tener un problema simple y poder de inicio identifcar de manera binaria si es positiva o negativa la review. Despues, querremos predecir la nota de la review por el texto con mas exactitud
dfSimpleBinary = dfCleaned.copy()
dfSimpleBinary['overall'] = dfSimpleBinary['overall'].apply(lambda x: 1 if x > 2.5 else 0)
dfSimpleBinary['overall'].value_counts()

dfSimpleBinary.head(1000)

,Unnamed: 0,overall,reviewText
0,0,1,issue
1,1,1,purchased device worked advertised never much ...
2,2,1,work expected sprung higher capacity think mad...
3,3,1,think worked greathad diff bran gb card went s...
4,4,1,bought retail packaging arrived legit orange e...
...,...,...,...
996,996,1,bought digital recorder memory card slot card ...
997,997,1,much say memory card work great go pro hero si...
998,998,1,gig microsd card cost less dollar per gig clas...
999,999,1,im nerd hoarder waited year memory gb wanted c...


In [11]:
#Guardamos el dataset limpio para no sobreescribirlo accidentalmente, este paso es solo una precaucion. solo falta borrar los nans
dfSimpleBinary.to_csv('amazon_reviews_simpleBinary.csv', index=False)

In [12]:
#applyCleantext

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Apply TF-IDF
tfidf = TfidfVectorizer(max_features=5000)  # Limit to 5000 features for simplicity
X = tfidf.fit_transform(dfSimpleBinary['reviewText'])  

# y = Etiqueta
y = dfSimpleBinary['overall']  

In [14]:
# Això nomes per ara per visualitzar el resultat despres de TF-IDF. X : sparse matrix (4914,5000), sparse matrix es representa per coordenades ja que en gran part està buida
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 103606 stored elements and shape (4914, 5000)>
  Coords	Values
  (0, 1665)	1.0
  (1, 2733)	0.24256964870629025
  (1, 824)	0.2216769033139674
  (1, 4818)	0.2236408225576124
  (1, 60)	0.312460206398619
  (1, 2083)	0.2409462699731393
  (1, 2038)	0.23215281367495225
  (1, 2314)	0.1548186128929903
  (1, 1939)	0.1653195812862317
  (1, 3522)	0.2672918307978223
  (1, 895)	0.3567246822937337
  (1, 1836)	0.23285639488699514
  (1, 3706)	0.3236595351873095
  (1, 349)	0.4727031839056542
  (2, 4812)	0.10477180298575468
  (2, 1094)	0.23846953938902
  (2, 1479)	0.3019338817065603
  (2, 427)	0.20836549227289494
  (2, 3916)	0.23021790712149928
  (2, 1858)	0.2575359655841075
  (2, 300)	0.2581013497957923
  (2, 941)	0.37234011189326427
  (2, 4492)	0.2478236895106856
  (2, 2245)	0.41974179272137446
  (2, 1825)	0.2622421303906132
  :	:
  (4912, 4587)	0.37343929726380504
  (4912, 3608)	0.2983013482906791
  (4912, 1200)	0.37913633663424273
  (4912,

In [15]:
print(y)

0       1
1       1
2       1
3       1
4       1
       ..
4910    0
4911    1
4912    1
4913    1
4914    1
Name: overall, Length: 4914, dtype: int64


In [16]:
import pandas as pd

# Cargar el dataset limpio
dfCleaned = pd.read_csv('amazon_reviews_Cleaned.csv')

# Crear el dataset ternario
dfTernary = dfCleaned.copy()
dfTernary['overall'] = dfTernary['overall'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))

# Verificar la distribución de las etiquetas
print(dfTernary['overall'].value_counts())

# Guardar el nuevo dataset
dfTernary.to_csv('amazon_reviews_ternary.csv', index=False)

print("Dataset ternario guardado como 'amazon_reviews_ternary.csv'")

2    4448
0     324
1     142
Name: overall, dtype: int64
Dataset ternario guardado como 'amazon_reviews_ternary.csv'
